<a href="https://colab.research.google.com/github/EzgiAlbardak/YZUE/blob/main/YZUE_Hafta3_NesneAlgilama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!nvidia-smi

Fri May 23 13:18:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             35W /   70W |     294MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [17]:
# Pip install method (recommended)

!pip install ultralytics==8.2.103 -q

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 41.6/112.6 GB disk)


In [21]:
from ultralytics import YOLO

from IPython.display import display, Image

In [20]:
from ultralytics import YOLO
import os

# Ağırlık dosyasının varsayılan olarak kaydedildiği dizini kontrol edelim
# (Genellikle ~/.cache/ultralytics/models/ veya benzeri)
# İlk çalıştırmada indirildiğini göreceksiniz.
# Eğer zaten indirilmişse, tekrar indirilmez.

print("YOLOv8 Nano modelini yüklüyorum...")
model_n = YOLO('yolov8n.pt') # Eğer yoksa, yolov8n.pt otomatik olarak indirilir.
print("YOLOv8 Nano modeli yüklendi.")

print("\nYOLOv8 Large modelini yüklüyorum...")
model_l = YOLO('yolov8l.pt') # Eğer yoksa, yolov8l.pt otomatik olarak indirilir.
print("YOLOv8 Large modeli yüklendi.")

# Ağırlıkların indirildiği klasörü kontrol edebilirsiniz (isteğe bağlı)
# Örneğin: print(os.path.expanduser('~/.cache/ultralytics/models'))

YOLOv8 Nano modelini yüklüyorum...
YOLOv8 Nano modeli yüklendi.

YOLOv8 Large modelini yüklüyorum...
YOLOv8 Large modeli yüklendi.


In [22]:
# Gerekli kütüphaneleri yükleyin
# !pip install ultralytics  # Yüklü değilse uncomment yapın
# !pip install opencv-python # Yüklü değilse uncomment yapın
# !pip install matplotlib    # Yüklü değilse uncomment yapın

# Bağımlılık sorunlarını gidermek için genellikle faydalıdır (NumPy hatası için)
# Eğer "numpy.dtype size changed" hatasını alırsanız, bu satırı çalıştırın ve Runtime'ı yeniden başlatın
!pip install numpy --upgrade --pre

# Kütüphaneleri içeri aktaralım
import os
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import numpy as np # Resim oluşturmak veya işlemek için
from google.colab import drive # Gerekirse drive mount etmek için

print("Kütüphaneler başarıyla yüklendi ve içe aktarıldı.")

  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ultralytics 8.2.103 requires numpy<2.0.0,>=1.23.0, but you have numpy 2.2.6 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
Kütüphaneler başarıyla yüklendi ve içe aktarıldı.


In [25]:
# Ortam kurulumunu yaptığınız varsayılıyor

# 1. YOLOv8 modelini yükleyin (örneğin, 'yolov8n.pt' - nano model)
# İlk çalıştırmada indirme mesajlarını görebilirsiniz.
model = YOLO('yolov8n.pt')
print(f"YOLOv8 modeli yüklendi: {model.model_name}")

YOLOv8 modeli yüklendi: yolov8n.pt


In [26]:
# Bu kısmı çalıştırmak için 'yolov8n-seg.pt' modelini indirmeniz gerekebilir
# !pip install ultralytics # Zaten yüklüyse çalıştırmayın
# import cv2
# import numpy as np
# from ultralytics import YOLO

from ultralytics import YOLO
import cv2

# Video ve model
cap = cv2.VideoCapture('traffic.mp4')
model = YOLO('yolov8n-seg.pt')

# Kontrol et
if not cap.isOpened():
    print("Video açılamadı!")
else:
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    print(f"Boyut: {width}x{height}, FPS: {fps}")

    # Video kaydedici
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

    # Kare kare işlem
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model.predict(source=frame, conf=0.5, show=False, save=False, verbose=False)
        annotated = results[0].plot() if results and results[0].masks is not None else frame

        out.write(annotated)

    cap.release()
    out.release()
    print("Video yazıldı: output.mp4")


Boyut: 1920x1080, FPS: 29.97002997002997
Video yazıldı: output.mp4


In [31]:
!ls -lh output.mp4

-rw-r--r-- 1 root root 293K May 23 13:23 output.mp4


In [33]:
from google.colab import files
files.download('output.mp4')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>